In [23]:
import numpy as np
from scipy.optimize import linprog

# Functions (linear programming)

In [36]:
#objecvec for men and women
def func_objecvec(umen, uwomen):
  men = len(umen)
  women = len(uwomen)
   
  objecvec = [0] * (men * women)

  #men
  for i in range(women):
    objecvec[-women + i] -= umen[-1][i]

  #women
  for i in range(men):
    objecvec[(i + 1) * women - 1] -= uwomen[-1][i]
  
  return objecvec

In [37]:
#constrains matrix
def func_constraints(umen, uwomen):
  men = len(umen)
  women = len(uwomen)

  rows = (men - 1) + (women - 1) + men + women
  columns = men * women
  constraints = np.zeros((rows, columns))

  #first (men - 1) men more than the last man
  for i in range(men - 1):
    temp = [0] * (men * women)
    for j in range(women):
      temp[-j - 1] = umen[-1][-j - 1]
    for j in range(women):
      temp[i * women + j] = -umen[i][j]
    constraints[i] = temp

  #first (women - 1) women more than the last woman
  for i in range(women - 1):
    temp = [0] * (men * women)
    for j in range(men):
      temp[women * (j + 1) - 1] = uwomen[-1][j]
    for j in range(men):
      temp[i + j * women] = -uwomen[i][j]
    constraints[men - 1 + i] = temp

  #men probability no more than 1
  for i in range(men):
    temp = [0] * (men * women)
    for j in range(women):
      temp[i * women + j] = 1
    constraints[rows - women - men + i, :] = temp

  #women probability no more than 1
  for i in range(women):
    temp = [0] * (women * men)
    for j in range(men):
      temp[j * women + i] = 1
    constraints[rows - women + i, :] = temp
  
  return constraints

In [38]:
def func_constraints_bounds(umen, uwomen):
  men = len(umen)
  women = len(uwomen)

  constraints_bounds = [0] * (men - 1 + women - 1 + men + women)
  
  for i in range(men + women):
    constraints_bounds[-i - 1] = 1

  return constraints_bounds

In [39]:
def func_boundslist(umen, uwomen):
  men = len(umen)
  women = len(uwomen)

  temp = (0, 1)
  boundslist = []
  
  for i in range (women * men):
    boundslist.append(temp)
  
  return boundslist

In [40]:
def func_expected_utility (umen, uwomen):
  men = len(umen)
  women = len(uwomen)

  objecvec = func_objecvec(umen, uwomen)
  constraints = func_constraints(umen, uwomen)
  constraints_bounds = func_constraints_bounds(umen, uwomen)
  boundslist = func_boundslist(umen, uwomen)
  
  res = linprog(objecvec, A_ub = constraints, b_ub = constraints_bounds, bounds = boundslist, method = 'revised simplex')
  result = res.x

  probabilities = np.zeros((men, women))

  for i in range(men * women):
    row = i // women
    column = i - row * women
    probabilities[row][column] = result[i]

  #men
  revmen = [0] * men
  for i in range(men):
    for j in range(women):
      revmen[i] += probabilities[i][j] * umen[i][j]

  #women
  revwomen = [0] * women
  for i in range(women):
    for j in range(men):
      revwomen[i] += probabilities[j][i] * uwomen[i][j]

  ans = len(revmen) * min(revmen) + len(revwomen) * min(revwomen)
  
  return ans, probabilities

# Functions (creating list of deleted from rand number)

In [29]:
#list from number for greedy algorithm
def func_delete_list_greedy(i):
  delete_list = []
  temp = 0

  while i > 0:
    if i % 2 == 1:
      delete_list.append(temp)
    i = i // 2
    temp += 1
    
  return delete_list

In [30]:
#list from number for sorted algorithm
def func_delete_list_sorted(i):
  delete_list = []

  for j in range(i):
    delete_list.append((-1) * (j + 1))
    
  return delete_list

# Functions (greedy and sorted algorithms, custom sorting function)

In [31]:
def func_greedy_eu(umen, uwomen):
  men = len(umen)
  women = len(uwomen)
  maximum_revenue = 0

  for i in range(2 ** men - 1):
    delete_men = func_delete_list_greedy(i)

    for j in range(2 ** women - 1):
      delete_women = func_delete_list_greedy(j)

      umen_temp = umen.copy()
      uwomen_temp = uwomen.copy()

      umen_temp = np.delete(umen_temp, delete_men, axis = 0)
      umen_temp = np.delete(umen_temp, delete_women, axis = 1)

      uwomen_temp = np.delete(uwomen_temp, delete_women, axis = 0)
      uwomen_temp = np.delete(uwomen_temp, delete_men, axis = 1)

      result = func_expected_utility(umen_temp, uwomen_temp)
      eu = result[0]
      probs = result[1]

      if maximum_revenue < eu:
        maximum_revenue = eu
        umen_max = umen_temp.copy()
        uwomen_max = uwomen_temp.copy()
        deleted_men_max = delete_men
        deleted_women_max = delete_women
        probabilities = probs

  return maximum_revenue, umen_max, uwomen_max, deleted_men_max, deleted_women_max, probabilities

In [32]:
def func_sorted_eu(umen, uwomen):
  men = len(umen)
  women = len(uwomen)
  maximum_revenue = 0

  for i in range(men):
    delete_men = func_delete_list_sorted(i)

    for j in range(women):
      delete_women = func_delete_list_sorted(j)

      umen_temp = umen.copy()
      uwomen_temp = uwomen.copy()

      umen_temp = np.delete(umen_temp, delete_men, axis = 0)
      umen_temp = np.delete(umen_temp, delete_women, axis = 1)

      uwomen_temp = np.delete(uwomen_temp, delete_women, axis = 0)
      uwomen_temp = np.delete(uwomen_temp, delete_men, axis = 1)

      result = func_expected_utility(umen_temp, uwomen_temp)
      eu = result[0]
      probs = result[1]

      if maximum_revenue < eu:
        maximum_revenue = eu
        umen_max = umen_temp.copy()
        uwomen_max = uwomen_temp.copy()
        deleted_men_max = delete_men
        deleted_women_max = delete_women
        probabilities = probs

  return maximum_revenue, umen_max, uwomen_max, deleted_men_max, deleted_women_max, probabilities

In [ ]:
def custom_sort(array_first, array_second):
  rows = len(array_first)
  for i in range(0, rows):
    for j in range(0, rows - i - 1):
      if max(array_first[j]) < max(array_first[j + 1]):

        #changing rows
        array_first[[j, j + 1]] = array_first[[j + 1, j]]

        #changing columns
        array_second[:, [j, j + 1]] = array_second[:, [j + 1, j]]

  return array_first, array_second

# Input

In [33]:
#insert number of men and women
men, women = map(int, input().split())

#insert matrix for men
umen = np.zeros((men, women))
for i in range(men):
  umen[i] = np.array(list(map(int, input().split())))

#insert matrix for women
uwomen = np.zeros((women, men))
for i in range(women):
  uwomen[i] = np.array(list(map(int, input().split())))


3 3
10 6 1
6 8 1
1 1 2
10 6 5
6 12 5
5 5 20


# Sorting

In [241]:
umen_raw = umen.copy()
uwomen_raw = uwomen.copy()

umen_sorted = umen.copy()
uwomen_sorted = uwomen.copy()

print('uwomen unsorted')
print(uwomen_raw, end = '\n\n')

print('umen unsorted')
print(umen_raw, end = '\n\n')

uwomen_sorted, umen_sorted = custom_sort(uwomen_sorted, umen_sorted)
umen_sorted, uwomen_sorted = custom_sort(umen_sorted, uwomen_sorted)

print('uwomen sorted')
print(uwomen_sorted, end = '\n\n')

print('umen sorted')
print(umen_sorted, end = '\n\n')

uwomen unsorted
[[10.  6.  5.]
 [ 6. 12.  5.]
 [ 5.  5. 20.]]

umen unsorted
[[10.  6.  1.]
 [ 6.  8.  1.]
 [ 1.  1.  2.]]

uwomen sorted
[[ 5.  5. 20.]
 [ 6. 12.  5.]
 [10.  6.  5.]]

umen sorted
[[ 1.  6. 10.]
 [ 1.  8.  6.]
 [ 2.  1.  1.]]



# Comparison of greedy and sorted

In [242]:
help_text = ['1. Maximum revenue', '2. Umen', '3. Uwomen', '4. Deleted men', '5. Deleted women', '6. Probabilities']

result_greedy = func_greedy_eu(umen_sorted, uwomen_sorted)
result_sorted = func_sorted_eu(umen_sorted, uwomen_sorted)

for i in range(len(help_text)):
  print(help_text[i])
  print(i + 1, ".", 1, " Greedy", sep = "")
  print(result_greedy[i])
  print(i + 1, ".", 2, " Sorted", sep = "")
  print(result_greedy[i])
  print()


1. Maximum revenue
1.1 Greedy
33.85714285714284
1.2 Sorted
33.85714285714284

2. Umen
2.1 Greedy
[[10.  6.  1.]
 [ 6.  8.  1.]
 [ 1.  1.  2.]]
2.2 Sorted
[[10.  6.  1.]
 [ 6.  8.  1.]
 [ 1.  1.  2.]]

3. Uwomen
3.1 Greedy
[[10.  6.  5.]
 [ 6. 12.  5.]
 [ 5.  5. 20.]]
3.2 Sorted
[[10.  6.  5.]
 [ 6. 12.  5.]
 [ 5.  5. 20.]]

4. Deleted men
4.1 Greedy
[]
4.2 Sorted
[]

5. Deleted women
5.1 Greedy
[]
5.2 Sorted
[]

6. Probabilities
6.1 Greedy
[[1.         0.         0.        ]
 [0.         0.71428571 0.        ]
 [0.         0.28571429 0.5       ]]
6.2 Sorted
[[1.         0.         0.        ]
 [0.         0.71428571 0.        ]
 [0.         0.28571429 0.5       ]]



# Sorted

In [ ]:
print('SORTED', end = '\n\n')
result = func_sorted_eu(umen, uwomen)
for i in range(len(result)):
  print(help_text[i])
  print(result[i])
  print()

SORTED

1. Maximum revenue
40.0

2. Umen
[[10.  6.]
 [ 6.  8.]]

3. Uwomen
[[20.  5.]
 [ 5. 12.]]

4. Deleted men
[-1]

5. Deleted women
[-1]

6. Probabilities
[[0.8 0. ]
 [0.  1. ]]

